# Quickstart


## Working with data
``torch.utils.data.Dataset``은 샘플과 해당 레이블을 저장하고``torch.utils.data.DataLoader`` 및  ``Dataset`` 주위에 iterable을 감쌉니다.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### Dataset 불러오기

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

``Dataset``을 ``DataLoader``에 대한 인수로 전달합니다. 이는 데이터 세트에 대해 반복 가능 항목을 래핑하고 자동 일괄 처리, 샘플링, 셔플링 및 다중 프로세스 데이터 로드를 지원합니다. 여기서 배치 크기를 64로 정의합니다. 

In [3]:
batch_size = 64

# data loader 생성
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


### 모델 만들기

In [4]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
device

'cpu'

In [5]:
# model 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Optimizer 지정

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            print(f"loss: {loss.item():>7f}")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298126
loss: 2.282128
loss: 2.258395
loss: 2.257514
loss: 2.238839
loss: 2.197508
loss: 2.216711
loss: 2.173959
loss: 2.166939
loss: 2.136526
Test Error: 
 Accuracy: 40.1%, Avg loss: 2.128548 

Epoch 2
-------------------------------
loss: 2.143155
loss: 2.126750
loss: 2.059357
loss: 2.082157
loss: 2.033741
loss: 1.962813
loss: 1.999861
loss: 1.912780
loss: 1.918629
loss: 1.847710
Test Error: 
 Accuracy: 59.0%, Avg loss: 1.839946 

Epoch 3
-------------------------------
loss: 1.883042
loss: 1.845286
loss: 1.715500
loss: 1.767129
loss: 1.651098
loss: 1.608344
loss: 1.636306
loss: 1.534652
loss: 1.566053
loss: 1.460836
Test Error: 
 Accuracy: 61.8%, Avg loss: 1.471867 

Epoch 4
-------------------------------
loss: 1.548944
loss: 1.508487
loss: 1.347756
loss: 1.432506
loss: 1.308789
loss: 1.309839
loss: 1.332294
loss: 1.254406
loss: 1.292598
loss: 1.194784
Test Error: 
 Accuracy: 64.3%, Avg loss: 1.216411 

Epoch 5
------------------------

### Model 저장

In [10]:
torch.save(model.state_dict(), "model.pth")

### 저장한 모델 불러오기

In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

모델을 이용한 예측

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
print(x.shape, y)

with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    
    predicted = classes[pred[0].argmax(0)]
    print(f'Predicted: "{predicted}", Actual: "{classes[y]}"')

torch.Size([1, 28, 28]) 9
Predicted: "Ankle boot", Actual: "Ankle boot"
